# Inference

In [2]:
from pathlib import Path
import os
os.environ["WANDB_NOTEBOOK_NAME"] = "xgboost_inference.ipynb"  # Manually set the notebook name

import polars as pl
import xgboost as xgb


import wandb
from tqdm.notebook import tqdm

In [3]:
# import math
# from pathlib import Path
# import os
# from typing import Callable

# import polars as pl
# import numpy as np
# import xgboost as xgb
# import optuna
# from optuna_integration.wandb import WeightsAndBiasesCallback
# from optuna_integration.xgboost import XGBoostPruningCallback
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# from sklearn.compose import TransformedTargetRegressor
# import shap

# import seaborn as sns
# import matplotlib.pyplot as plt
# import plotly.express as px

# import wandb
# from wandb.integration.xgboost import WandbCallback
# from tqdm.notebook import tqdm

In [4]:
DEBUG = True

In [11]:
config = {}

In [12]:
# Choose training run from which to load the model, etc.
train_run_name = 'helpful-flower-65'

In [13]:
run = wandb.init(project="traffic-forecasting-challenge", tags=[train_run_name], job_type='inference', entity="esedx12", config=config, save_code=True, mode=('dryrun' if DEBUG else 'online'))

wandb: Network error resolved after 0:13:45.244177, resuming normal operation.


In [15]:
checkpoints_dir = 'checkpoints'
xgboost_models_dir = Path(checkpoints_dir) / train_run_name

models = {}
for file_name in os.listdir(xgboost_models_dir):
    if file_name.endswith('.ubj'):
        model_name = file_name[:-4]
        model_path = xgboost_models_dir / file_name
        models[model_name] = xgb.Booster(model_file=str(model_path))

wandb: Network error (ConnectionError), entering retry loop.


## Load and prepare data

In [ ]:
# Read the CSV files
data_dir = Path('input-data')
thp_vol = pl.read_csv(data_dir / 'traffic_DLThpVol.csv')  # This is the target variable
prb = pl.read_csv(data_dir / 'traffic_DLPRB.csv')
thp_time = pl.read_csv(data_dir / 'traffic_DLThpTime.csv')
mr_number = pl.read_csv(data_dir / 'traffic_MR_number.csv')

target_dataframes = {
    'thp_vol': thp_vol,
    'prb': prb,
    'thp_time': thp_time,
    'mr_number': mr_number
}

# Rename first col to 'hour'
for k, v in target_dataframes.items():
    target_dataframes[k] = v.rename({'': "idx_hour"})

## Multi-Step Inference

In [ ]:
def predict_one_step(target_dataframes: dict[pl.DataFrame], model: xgb.Booster, config: wandb.Config) -> dict[pl.DataFrame]:
    """
    Predict one step into the future using a trained model.
    Takes DataFrames of len n, returns DataFrames of len n + 1.
    """
    template_df = next(iter(target_dataframes.values()))
    predict_hour = template_df['idx_hour'][-1] + 1

    null_row = pl.DataFrame({col: [None] if not col == 'idx_hour' else predict_hour for col in template_df.columns})
    target_dataframes = {k: pl.concat([v, null_row], how='vertical_relaxed') for k, v in target_dataframes.items()}

    target_names = list(target_dataframes.keys())
    feature_dfs = create_all_feature_dfs(target_dataframes, config)
    feature_dfs = {k: v.tail(1) for k, v in feature_dfs.items()}  # maybe turn in to lazyframe for efficiency?
    X_predict = convert_to_long_format(feature_dfs)
    idx_hour, beam_id = X_predict['idx_hour'], X_predict['beam_id']
    X_predict = X_predict.drop(dropped_cols)

    # We predict only the idx immediately folling the last idx in the input, ie a single row
    y_predicted_long = model.predict(X_predict)
    y_predicted_long = pl.DataFrame(y_predicted_long, schema=target_names).with_columns([idx_hour, beam_id])
    y_predicted_wide = convert_to_wide_format(y_predicted_long, output_df_names=target_names)

    return {target_name: pl.concat([target_dataframes[target_name].head(-1), y_predicted_wide[target_name]], how='vertical_relaxed') for target_name in target_names}

In [ ]:
def predict_multi_step(target_dataframes: dict[pl.DataFrame], model: xgb.Booster, config: wandb.Config, num_steps: int, max_lag=None) -> dict[pl.DataFrame]:
    """
    Predict multiple steps into the future using a trained model.
    Takes DataFrames of len n, returns DataFrames of len n + num_steps.
    """
    if max_lag:
        target_dataframes = {k: v.tail(max_lag + 5) for k, v in target_dataframes.items()}

    for _ in tqdm(range(num_steps)):
        target_dataframes = predict_one_step(target_dataframes, model, config)

    return target_dataframes

In [ ]:
long_train_df = create_long_train_df(target_dataframes, wandb.config)
long_test_df = create_long_train_df(test_dataframes, wandb.config)

dropped_cols = ['idx_hour', 'beam_id']
target_cols = list(target_dataframes.keys())

X_train, y_train = long_train_df.drop(dropped_cols + target_cols), long_train_df.select(target_cols)
X_test, y_test = long_test_df.drop(dropped_cols + target_cols), long_test_df.select(target_cols)

wandb.log({'train shape': X_train.shape, 'test shape': X_test.shape, 'train_feats': X_train.columns,})

In [ ]:
y = predict_multi_step({k: v.tail(80) for k, v in train_dataframes.items()}, model, wandb.config, num_steps=len(test_dataframes['thp_vol'])).mean_horizontal()

  0%|          | 0/120 [00:00<?, ?it/s]

/tmp/ipykernel_2432706/3820107680.py:19: DeprecationWarning:

The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.



AttributeError: 'dict' object has no attribute 'mean_horizontal'

In [ ]:
def mean_absolute_error(Y_true: pl.DataFrame, Y_pred: pl.DataFrame) -> float:
    """
    Compute the mean absolute error between two DataFrames.
    """
    assert (Y_true['idx_hour'] == Y_pred['idx_hour']).all(), "DataFrames must be aligned"

    return (Y_true.drop('idx_hour') - Y_pred.drop('idx_hour')).select(pl.all().abs().mean()).mean_horizontal()[0]

In [ ]:
mean_absolute_error(test_dataframes['thp_vol'], y['thp_vol'].tail(len(test_dataframes['thp_vol'])))

0.24911949991279875

## ...on Validation Set

## ...on Test Set

## ...on Validation and Test Sets

## Create Submission CSV

* Hours in 5 weeks: 840
* Hours in 6 weeks: 1008
* We need period 841-1008 (841-1009 with Python list indexing)

* Hours in 10 weeks: 1680
* Hours in 11 weeks: 1848

In [ ]:
def create_half_submission_df(input_df: pl.DataFrame, weeks: str) -> pl.DataFrame:
    """
    Create a submission CSV file from a Polars DataFrame.
    """
    if weeks == '5w-6w':
        range = [841, 1008]
    elif weeks == '10w-11w':
        range = [1681, 1848]

    # Choose rows with first column 'idx_hour' having the values 671-840.
    input_df = input_df.filter(pl.col('idx_hour').is_in(range)).with_row_index()

    # Check that shape of dataframe is (168, 2881)
    assert input_df.shape == (168, 2881), f"Expected shape (168, 2881), got {input_df.shape}"

    # Check that there is no null value in the dataframe
    assert input_df.is_null().any().any() == False, "Submission dataframe contains null values"

    # Stack the dataframe with f'traffic_DLThpVol_test_5w-6w_{hour}_{beam_id}' as index
    # where it cycles through the values 671-840 for hour and then the beam_ids, which are colnames of input_df
    return input_df.unpivot(index='idx_hour').with_columns(
        (pl.struct(pl.all()).map_elements(lambda row: f'traffic_DLThpVol_test_5w-6w_{row['row_index']}_{row["variable"]}', return_dtype=pl.String)).alias('ID')
    ).select(['ID', 'value']).rename({'value': 'Target'})


def create_submission_csv(input_df: pl.DataFrame, output_filename='traffic_forecast.csv', archiving_dir='submission-csvs-archive') -> pl.DataFrame:
    """
    Create a submission CSV file from data in input format that's been extended to cover weeks 5-6 and 10-11.
    """

    # Create half submission dataframes
    half_submission_5w_6w = create_half_submission_df(input_df, '5w-6w')
    half_submission_10w_11w = create_half_submission_df(input_df, '10w-11w')

    # Concatenate the two half submission dataframes
    submission_df = pl.concat([half_submission_5w_6w, half_submission_10w_11w], how='vertical')

    # Save the submission dataframe to a CSV file for submission
    submission_df.write_csv(output_filename)
    
    # Save the submission dataframe to a CSV file for archiving
    if archiving_dir:
        archiving_dir = Path(archiving_dir)
        archiving_dir.mkdir(parents=True, exist_ok=True)
        submission_df.write_csv(archiving_dir / f'{wandb.run.name}_{output_filename}')

    return submission_df